# --- Day 15: Chiton --- 

https://adventofcode.com/2021/day/15

## Get Input Data

In [17]:
def parse_data(filename):
    """Read in risk map data."""

    with open(f'../inputs/{filename}') as file:
        risk_map = [[int(x) for x in line.strip()] for line in file.readlines()]
    
    risk_dict = {}

    for i in range(len(risk_map[0])):
        for j in range(len(risk_map)):
            risk_dict[(i, j)] = risk_map[i][j]

    # We're gonna start at the bottom right corner when we search for the min risk path
    start_position = (len(risk_map[0])-1, len(risk_map)-1)

    return risk_map, risk_dict, start_position

In [19]:
test_risk_map, test_risk_dict, test_start_position = parse_data('test_risk_map.txt')
print(f'test risk map:\n {test_risk_map}\n')
print(f'test risk dict:\n {test_risk_dict}\n')
print(f'test start position: {test_start_position}')

test risk map:
 [[1, 1, 6, 3, 7, 5, 1, 7, 4, 2], [1, 3, 8, 1, 3, 7, 3, 6, 7, 2], [2, 1, 3, 6, 5, 1, 1, 3, 2, 8], [3, 6, 9, 4, 9, 3, 1, 5, 6, 9], [7, 4, 6, 3, 4, 1, 7, 1, 1, 1], [1, 3, 1, 9, 1, 2, 8, 1, 3, 7], [1, 3, 5, 9, 9, 1, 2, 4, 2, 1], [3, 1, 2, 5, 4, 2, 1, 6, 3, 9], [1, 2, 9, 3, 1, 3, 8, 5, 2, 1], [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]]

test risk dict:
 {(0, 0): 1, (0, 1): 1, (0, 2): 6, (0, 3): 3, (0, 4): 7, (0, 5): 5, (0, 6): 1, (0, 7): 7, (0, 8): 4, (0, 9): 2, (1, 0): 1, (1, 1): 3, (1, 2): 8, (1, 3): 1, (1, 4): 3, (1, 5): 7, (1, 6): 3, (1, 7): 6, (1, 8): 7, (1, 9): 2, (2, 0): 2, (2, 1): 1, (2, 2): 3, (2, 3): 6, (2, 4): 5, (2, 5): 1, (2, 6): 1, (2, 7): 3, (2, 8): 2, (2, 9): 8, (3, 0): 3, (3, 1): 6, (3, 2): 9, (3, 3): 4, (3, 4): 9, (3, 5): 3, (3, 6): 1, (3, 7): 5, (3, 8): 6, (3, 9): 9, (4, 0): 7, (4, 1): 4, (4, 2): 6, (4, 3): 3, (4, 4): 4, (4, 5): 1, (4, 6): 7, (4, 7): 1, (4, 8): 1, (4, 9): 1, (5, 0): 1, (5, 1): 3, (5, 2): 1, (5, 3): 9, (5, 4): 1, (5, 5): 2, (5, 6): 8, (5, 7): 1, (5, 8)

In [ ]:
risk_map, risk_dict, start_position = parse_data('risk_map.txt')

## Part 1
---

Need to take a dynamic programming approach and solve from the end to the beginning to find the path with the least risk.

In [44]:
def get_neighbors(current_position, start_position, visited):
    """Return a list of possible next moves; can only move horizontally or vertically, and
    we don't want to allow for any backtracking.
    """

    y, x = current_position[0], current_position[1]
    neighbors = [(y+1, x), (y-1, x), (y, x+1), (y, x-1)]
    
    max_height, max_length = start_position[0], start_position[1]

    # Filter out neighbors who have gone over the edge!
    good_neighbors = list(filter(lambda n: 0 <= n[0] <= max_height and 0 <= n[1] <= max_length, neighbors))

    # Also filter out neighbors if they aren't already in the visited list
    good_neighbors = list(set(good_neighbors) - set(visited))

    return good_neighbors

In [48]:
def make_move(current_position, start_position, risk_dict, visited):
    """Return the position of the next move along the path, which will be the horizontal or
    vertical neighbor (that hasn't already been visited) with the lowest risk score.
    """

    good_neighbors = get_neighbors(current_position, start_position, visited)
    print(f'current position: {current_position}')
    print(f'good neighbors: {good_neighbors}')
    next_position = min(good_neighbors, key=risk_dict.__getitem__)
    print(f'best neighbor: {next_position}, with a risk of {risk_dict[next_position]}')

    return next_position

In [51]:
def find_path_with_minimum_risk(position, start_position, risk_dict):
    """Reverse through the risk map from the end position and find the path with the least risk;
    Return the total risk for the path.
    """
    
    visited = [position]
    total_risk = risk_dict[position]

    while position != (0, 0):
        print(f'visited so far: {visited}')
        print(f'total risk so far: {total_risk}\n')
        position = make_move(position, start_position, risk_dict, visited)
        visited.append(position)
        total_risk += risk_dict[position]

    return total_risk

### Run on Test Data

In [52]:
find_path_with_minimum_risk(test_start_position, test_start_position, test_risk_dict)  # Should return 40

visited so far: [(9, 9)]
total risk so far: 1

current position: (9, 9)
good neighbors: [(8, 9), (9, 8)]
best neighbor: (8, 9), with a risk of 1
visited so far: [(9, 9), (8, 9)]
total risk so far: 2

current position: (8, 9)
good neighbors: [(7, 9), (8, 8)]
best neighbor: (8, 8), with a risk of 2
visited so far: [(9, 9), (8, 9), (8, 8)]
total risk so far: 4

current position: (8, 8)
good neighbors: [(8, 7), (9, 8), (7, 8)]
best neighbor: (7, 8), with a risk of 3
visited so far: [(9, 9), (8, 9), (8, 8), (7, 8)]
total risk so far: 7

current position: (7, 8)
good neighbors: [(7, 9), (6, 8), (7, 7)]
best neighbor: (6, 8), with a risk of 2
visited so far: [(9, 9), (8, 9), (8, 8), (7, 8), (6, 8)]
total risk so far: 9

current position: (6, 8)
good neighbors: [(6, 7), (6, 9), (5, 8)]
best neighbor: (6, 9), with a risk of 1
visited so far: [(9, 9), (8, 9), (8, 8), (7, 8), (6, 8), (6, 9)]
total risk so far: 10

current position: (6, 9)
good neighbors: [(7, 9), (5, 9)]
best neighbor: (5, 9), wi

ValueError: min() arg is an empty sequence

### Run on Input Data

## Part 2
---

### Run on Test Data

### Run on Input Data